(Lecture8_Part2_SPH_Algorithm_For_Energy_Equation)=
# Lecture8_Part2_SPH_Algorithm_For_Energy_Equation

Open Anaconda Prompt (anaconda 3), then type

pip install ipyvolume

pip install bqplot

jupyter nbextension enable –py bqplot

In [39]:
import ipyvolume as ipv
import bqplot.scales
import numpy as np
from numpy import random
import ipywidgets as widgets    # https://www.youtube.com/watch?v=hOKa8klJPyo
import matplotlib.cm as cm    # for coloring temperature

In [40]:
G = np.array([0.0,-9.8,0.0]); # external (gravitational) forces
REST_DENS = 1000.0; # rest density of liquid
GAS_CONST = 2000.0; # const for equation of state
H = 16.0; # kernel radius
HSQ = H*H; # radius^2 for optimization
T0 = 1.0    # initial temperature of the fluid
T_bottom = 1.5    # bottom heating temperature
T_ceiling = 0.5    # upper cooling temperature
alpha_v = 0.0002    # thermal expansion coefficient
k = 0.0003    # thermal conductivity of the particle
k_wall = 100.0    # thermal conductivity of the wall
MASS = 65.0; # assume all particles have the same mass
VISC = 250.0; # viscosity constant
dt = 0.0008; # integration timestep

# smoothing kernels defined in Müller and their gradients
POLY6 = 315.0/(65.0*np.pi*H**9);
SPIKY_GRAD = -45.0/(np.pi*H**6);
VISC_LAP = 45.0/(np.pi*H**6);

# simulation parameters
EPS = H; # boundary epsilon
BOUND_DAMPING_NORMAL = -0.0;    # particle does not bounce back vertically
BOUND_DAMPING_HORIZONTAL = 0.0;    # no slip-boundary

# solver data
Fluid_Particles_list = [];
X = [[],[],[]]    # positions for all particles
Ts = []    # temperature of all particles

# interaction
MAX_PARTICLES = 3000;

# rendering projection parameters
WINDOW_WIDTH = 425;
WINDOW_HEIGHT = 150;
WINDOW_DEPTH = 300

VIEW_WIDTH = 1.5*WINDOW_WIDTH;
VIEW_HEIGHT = 1.5*WINDOW_HEIGHT;
VIEW_DEPTH = 1.5*WINDOW_DEPTH;


In [41]:
# particle data structure
# stores position, velocity, and force for integration
# stores density (rho) and pressure values for SPH
class Particle:
    def __init__(self, X, V, F, rho, p, T):
        self.X = X   # x position vector

        self.V = V    # velocity vector
      
        self.F = F    # force vector
       
        self.rho = 0.0    # density of the fluid at the position of the particle
        self.p = p    # pressure of the fluid at the position of the particle
        self.T = T0    # temperature of the fluid at the position of the particle


In [42]:
def InitSPH():
    global X
    global fig
    global scatter
    global ipv
    global Ts
    
    #for zi in range(int(2*EPS),int(WINDOW_DEPTH-EPS*2.0),int(H)):
        #zi += H
    for yi in range(int(1*EPS), int(WINDOW_HEIGHT-EPS*1.0), int(H*1/16)):
        yi += H #+random.rand()*H/1000
        for xi in range(0, int(WINDOW_WIDTH), int(H)):
            if len(Fluid_Particles_list) < MAX_PARTICLES:
                xi += H #+random.rand()*H/1000

                Vx = 0.0    # gives the particle a random tiny initial velocity in the x-direction
                Vy = 0.0    # gives the particle a random tiny initial velocity in the y-direction
                Vz = 0.0
                V = np.array([Vx,Vy,Vz])
                
                Fx = 0.0    # gives the particle a random tiny initial force in the x-direction
                Fy = 0.0    # gives the particle a random tiny initial force in the y-direction
                Fz = 0.0
                F = np.array([Fx,Fy,Fz])
                
                rho = 0.0
                p = 0.0
                T = T0

                zi = 2*EPS
                
                Xi = np.array([xi + np.random.random_sample(),yi + np.random.random_sample(),zi])# + np.random.random_sample()
                
                Fluid_Particles_list.append(Particle(Xi,V,F,rho,p,T))

    for pi in Fluid_Particles_list:
          
        X[0].append(pi.X[0])
        X[1].append(pi.X[1])
        X[2].append(pi.X[2])
        Ts.append(np.array([pi.T,pi.T,pi.T])) 
        
    x = np.array(X[0])
    y = np.array(X[1])
    z = np.array(X[2])
    
    # the scale does not seem to work...
    scales = {
    'x': bqplot.scales.LinearScale(min=0, max=VIEW_WIDTH),
    'y': bqplot.scales.LinearScale(min=0, max=VIEW_HEIGHT),
    'z': bqplot.scales.LinearScale(min=0, max=VIEW_DEPTH),
}
    
    fig = ipv.figure(scales=scales)
    Ts = np.array(Ts)
    Ts -= Ts.min()
    Ts /= Ts.max()
    colors = Ts
    scatter = ipv.scatter(x,y,z,marker='sphere',color=colors)
    
    scatter.size = 3
    
    ipv.show()
    print("initializing dam break with " )
    print(len(x))
    print("particles")

In [43]:
def ComputeDensityPressureTemperature():
    
    for pi in Fluid_Particles_list:
        pi.rho = 0.0
        dT = 0.0
        for pj in Fluid_Particles_list:
            h = np.linalg.norm(pi.X - pj.X)
            if (h<H):
                pi.T += (pj.T - pi.T) * k * dt *POLY6*(HSQ-h**2)**3;
                pi.rho += MASS*POLY6*(HSQ-h**2)**3;
        
        pi.rho -= pi.rho * alpha_v *(pi.T - T0)
        pi.p = GAS_CONST*(pi.rho - REST_DENS) * pi.T;

In [44]:
def ComputeForces():
    
    for pi in Fluid_Particles_list:
        
        fpress = 0.0
        fvisc = 0.0

        for pj in Fluid_Particles_list:
            
            d = pj.X - pi.X
            h = np.linalg.norm(d)
            r = d/h

            
            if (pi==pj):
                continue;
            
            if (h<H):
                
                # compute pressure force contribution
                fpress += - r * MASS * (pi.p + pj.p) / (2*pj.rho)  * SPIKY_GRAD * (H-h)**2
              
                # compute viscosity force contribution
                fvisc += VISC * MASS * (pj.V - pi.V) / (pj.rho)  * VISC_LAP * (H-h);
            

        fgrav = G*pi.rho
        pi.F = fpress + fvisc + fgrav    

In [45]:
def Integrate():
    for p in Fluid_Particles_list:
        # forward Euler integration
        p.V += dt*(p.F)/p.rho;
        
        p.V[2] = 0.0
        
        p.X += dt*p.V;
  
        if (p.X[0] - EPS < 0.0):    # left boundary condition
            p.V[0] *= 1.0 #BOUND_DAMPING_NORMAL
            p.X[0] = WINDOW_WIDTH - EPS    # coming out from the right boundary
            p.V[1] *= BOUND_DAMPING_HORIZONTAL
            #p.Vz *= BOUND_DAMPING_HORIZONTAL
        
        if (p.X[0] + EPS > WINDOW_WIDTH):    # right boundary condition
            p.V[0] *= 1.0 #BOUND_DAMPING_NORMAL
            p.X[0] = EPS    # coming out from the left boundary
            p.V[1] *= BOUND_DAMPING_HORIZONTAL
            #p.Vz *= BOUND_DAMPING_HORIZONTAL

        if (p.X[1] - EPS < 0.0):    # bottom boundary condition
            p.V[1] *= BOUND_DAMPING_NORMAL
            p.X[1] = EPS
            p.V[0] *= BOUND_DAMPING_HORIZONTAL
            
            p.T += (T_bottom - p.T) * k_wall * dt    # heating from the bottom
            #p.Vz *= BOUND_DAMPING_HORIZONTAL
            
        if (p.X[1] + EPS > WINDOW_HEIGHT):    # upper boundary condition
            p.V[1] *= BOUND_DAMPING_NORMAL
            p.X[1] = WINDOW_HEIGHT - EPS 
            p.V[0] *= BOUND_DAMPING_HORIZONTAL
            
            p.T += (T_ceiling - p.T) * k_wall * dt    # cooling from the ceiling
            #p.Vz *= BOUND_DAMPING_HORIZONTAL
         
        # only for 3D boundary condition (in the z-direction)
        if (p.X[2] - EPS < 0.0):    # front boundary condition
            p.V[2] *= BOUND_DAMPING_NORMAL
            p.X[2] = EPS
            #p.Vx *= BOUND_DAMPING_HORIZONTAL
            #p.Vy *= BOUND_DAMPING_HORIZONTAL
            
        if (p.X[2] + EPS > WINDOW_DEPTH):    # back boundary condition
            p.V[2] *= BOUND_DAMPING_NORMAL
            p.X[2] = WINDOW_DEPTH - EPS 
            #p.Vx *= BOUND_DAMPING_HORIZONTAL
            #p.Vy *= BOUND_DAMPING_HORIZONTAL

In [46]:
def Update():
    print("start updating")
    ComputeDensityPressureTemperature()
    ComputeForces()
    Integrate()    
    x =[]
    y =[]
    z =[]
    Ts = []
    for pi in Fluid_Particles_list:
        x.append(pi.X[0])
        y.append(pi.X[1])
        z.append(pi.X[2])
        Ts.append(np.array([pi.T,pi.T,pi.T])) 
    
    for n in Ts:
        max_ = 1.5
        min_ = 1.5
        temperature = n[0]
        if (temperature > max_):
            max_ = temperature
        if (temperature < min_):
            min_ = temperature

            
    Ts = np.array(Ts)
    Ts -= Ts.min()
    Ts /= Ts.max()
    colors = Ts
    scatter.x=np.array(x)
    scatter.y=np.array(y)
    scatter.z=np.array(z)
    scatter.color=colors

In [47]:
InitSPH()

C:\Users\JianouJiang\anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide


initializing dam break with 
3000
particles


In [ ]:
TotalTime = 1000
iterations = int(TotalTime/dt)
print("start iterating")
for time in range(iterations):
    Update()

    print("iterations")
    print(time)
        

start iterating
start updating
